In [1]:
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import math
from os import path
from keras.models import Sequential
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from numpy import hstack
import tensorflow as tf
import numpy as np
from collections import Counter
%matplotlib inline

In [2]:
DATA_DIR = "../../data"

# Load Data

In [3]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train_data.csv"), index_col=0)
chen_train.head()

,Antibody_ID,heavy,light,Y
2073,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0
1517,4yny,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,EFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2025,5xcv,EVQLVESGGGLVQPGRSLKLSCAASGFTFSNYGMAWVRQTPTKGLE...,QFVLTQPNSVSTNLGSTVKLSCKRSTGNIGSNYVNWYQQHEGRSPT...,1
2070,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1
666,2xqy,QVQLQQPGAELVKPGASVKMSCKASGYSFTSYWMNWVKQRPGRGLE...,DIVLTQSPASLALSLGQRATISCRASKSVSTSGYSYMYWYQQKPGQ...,0


In [4]:
chen_valid = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_valid_data.csv"), index_col=0)
chen_test = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test_data.csv"), index_col=0)
chen_valid = pd.concat([chen_valid, chen_test])
chen_valid.head()

,Antibody_ID,heavy,light,Y
2169,6ct7,EVQLVESGGGLVEPGGSLRLSCAVSGFDFEKAWMSWVRQAPGQGLQ...,SYELTQPPSVSVSPGQTARITCSGEALPMQFAHWYQQRPGKAPVIV...,0
1342,4nzu,AVSLVESGGGTVEPGSTLRLSCAASGFTFGSYAFHWVRQAPGDGLE...,DIEMTQSPSSLSASTGDKVTITCQASQDIAKFLDWYQQRPGKTPKL...,0
1728,5i8c,QEVLVQSGAEVKKPGASVKVSCRAFGYTFTGNALHWVRQAPGQGLE...,DIQLTQSPSFLSASVGDKVTITCRASQGVRNELAWYQQKPGKAPNL...,1
1729,5i8e,QEVLVQSGAEVKKPGASVKVSCRAFGYTFTGNALHWVRQAPGQGLE...,IQLTQSPSFLSASVGDKVTITCRASQGVRNELAWYQQKPGKAPNLL...,0
2114,6bb4,QVQLQQSDAELVKPGASVKISCKASGYTFTDRTIHWVKQRPEQGLE...,DVQMIQSPSSLSASLGDIVTMTCQASQDTSINLNWFQQKPGKAPKL...,0


In [5]:
x_chen = pd.read_csv(path.join(DATA_DIR, "chen/integer_encoding/chen_integer_encoded.csv"), index_col=0)
x_chen_train = x_chen.loc[chen_train.index]
x_chen_test = x_chen.loc[chen_valid.index]
x_chen_train.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,271,272,273,274,275,276,277,278,279,280
2073,6aod,4,18,14,10,18,14,16,6,1,...,0,0,0,0,0,0,0,0,0,0
1517,4yny,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2025,5xcv,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
2070,6and,4,18,14,10,18,4,16,6,6,...,0,0,0,0,0,0,0,0,0,0
666,2xqy,14,18,14,10,14,14,13,6,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = x_chen_train.drop("Ab_ID", axis=1).to_numpy()
X_test = x_chen_test.drop("Ab_ID", axis=1).to_numpy()

In [7]:
y_train = chen_train["Y"].to_numpy()
y_test = chen_valid["Y"].to_numpy()

In [8]:
#train_ohe = to_categorical(X_train)
#test_ohe = to_categorical(X_test)
#train_ohe[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
#train_ohe[0].shape

(281, 21)

In [10]:
max_length = len(train_ohe[0])
print(max_length)

281


# Models

### Model 1

In [11]:
model = Sequential()
model.add(LSTM(
    50, activation='sigmoid', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(50, activation='sigmoid', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

2021-12-20 09:58:35.298624: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 281, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [75]:
model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=128)

Epoch 1/200
11/11 [==============================] - 6s 317ms/step - loss: 3.2986
Epoch 2/200
11/11 [==============================] - 3s 317ms/step - loss: 2.8671
Epoch 3/200
11/11 [==============================] - 4s 319ms/step - loss: 2.5602
Epoch 4/200
11/11 [==============================] - 3s 308ms/step - loss: 2.3076
Epoch 5/200
11/11 [==============================] - 3s 316ms/step - loss: 2.1045
Epoch 6/200
11/11 [==============================] - 3s 316ms/step - loss: 1.9381
Epoch 7/200
11/11 [==============================] - 4s 320ms/step - loss: 1.8027
Epoch 8/200
11/11 [==============================] - 4s 317ms/step - loss: 1.6919
Epoch 9/200
11/11 [==============================] - 3s 318ms/step - loss: 1.6005
Epoch 10/200
11/11 [==============================] - 3s 311ms/step - loss: 1.5245
Epoch 11/200
11/11 [==============================] - 3s 313ms/step - loss: 1.4613
Epoch 12/200
11/11 [==============================] - 3s 311ms/step - loss: 1.4079
Epoch 13/200


In [80]:
model.evaluate(X_test, y_test, batch_size=1)

239/239 [==============================] - 7s 29ms/step - loss: 0.5075


0.5074639916419983

In [81]:
y_pred = model.predict(X_test)
y_pred

array([[0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.20581347],
       [0.205

Apparently all the predictions are the same :( Quite understandable, when the dataset is imbalanced. What to do about it?

In [ ]:
print(metrics.f1_score(y_test, y_pred))
print(metrics.accuracy_score(y_valid, y_pred))

### Model 2

In [17]:
model = Sequential()
model.add(LSTM(
    50, activation='relu', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(50, activation='relu', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 281, 50)           10400     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=64)

Epoch 1/200
21/21 [==============================] - 7s 244ms/step - loss: 612956672.0000
Epoch 2/200
21/21 [==============================] - 5s 236ms/step - loss: 3.4039
Epoch 3/200
21/21 [==============================] - 5s 238ms/step - loss: 3.3552
Epoch 4/200
21/21 [==============================] - 5s 245ms/step - loss: 3.3202
Epoch 5/200
21/21 [==============================] - 5s 237ms/step - loss: 3.2948
Epoch 6/200
21/21 [==============================] - 5s 241ms/step - loss: 3.2754
Epoch 7/200
21/21 [==============================] - 5s 247ms/step - loss: 3.2599
Epoch 8/200
21/21 [==============================] - 5s 236ms/step - loss: 3.2471
Epoch 9/200
21/21 [==============================] - 5s 240ms/step - loss: 3.2353
Epoch 10/200
21/21 [==============================] - 5s 241ms/step - loss: 3.2236
Epoch 11/200
21/21 [==============================] - 5s 237ms/step - loss: 3.2103
Epoch 12/200
21/21 [==============================] - 5s 239ms/step - loss: 3.1900
Epoch

KeyboardInterrupt: 

In [20]:
model.evaluate(X_test, y_test, batch_size=1)

239/239 [==============================] - 7s 29ms/step - loss: 3.0515


3.0515201091766357

In [21]:
y_pred = model.predict(X_test)
y_pred

array([[0.20992106],
       [0.20791924],
       [0.20984468],
       [0.21016249],
       [0.21216676],
       [0.21036375],
       [0.21095589],
       [0.21002048],
       [0.20772734],
       [0.20795539],
       [0.21061528],
       [0.21096697],
       [0.19717345],
       [0.19761881],
       [0.21084765],
       [0.211494  ],
       [0.21031821],
       [0.21067479],
       [0.20984936],
       [0.21006978],
       [0.20541656],
       [0.20973724],
       [0.20973724],
       [0.21127683],
       [0.21076229],
       [0.21076572],
       [0.20804015],
       [0.18784446],
       [0.21132302],
       [0.21070066],
       [0.2109569 ],
       [0.21123934],
       [0.20929185],
       [0.21117595],
       [0.21117586],
       [0.21108863],
       [0.21083713],
       [0.21036819],
       [0.2104238 ],
       [0.21092072],
       [0.21110901],
       [0.21186692],
       [0.20862168],
       [0.21096504],
       [0.20865539],
       [0.20658135],
       [0.1993657 ],
       [0.210

### Model 3

In [50]:
model = Sequential()
model.add(LSTM(200, activation="relu", input_shape=(max_length, 1), return_sequences=True))  
model.add(Dropout(0.2))  
model.add(LSTM(200, activation="relu", return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [52]:
model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=128)

Epoch 1/200
11/11 [==============================] - 33s 3s/step - loss: 9.4642
Epoch 2/200
11/11 [==============================] - 33s 3s/step - loss: 11.9568
Epoch 3/200
11/11 [==============================] - 33s 3s/step - loss: 12.0353
Epoch 4/200
11/11 [==============================] - 33s 3s/step - loss: 12.0467
Epoch 5/200
11/11 [==============================] - 33s 3s/step - loss: 12.0467
Epoch 6/200
11/11 [==============================] - 32s 3s/step - loss: 12.0353
Epoch 7/200
11/11 [==============================] - 33s 3s/step - loss: 12.0353
Epoch 8/200
11/11 [==============================] - 33s 3s/step - loss: 12.0354
Epoch 9/200
11/11 [==============================] - 33s 3s/step - loss: 12.0467
Epoch 10/200
11/11 [==============================] - 33s 3s/step - loss: 12.0467
Epoch 11/200
11/11 [==============================] - 32s 3s/step - loss: 12.0353
Epoch 12/200
11/11 [==============================] - 33s 3s/step - loss: 12.0353
Epoch 13/200
11/11 [======

KeyboardInterrupt: 

### Model 4

In [55]:
model = Sequential()
model.add(LSTM(200, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(Dropout(0.1))  
model.add(LSTM(200, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [56]:
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
11/11 [==============================] - 33s 3s/step - loss: 11.3796
Epoch 2/100
11/11 [==============================] - 31s 3s/step - loss: 8.6393
Epoch 3/100
11/11 [==============================] - 31s 3s/step - loss: 7.1998
Epoch 4/100
11/11 [==============================] - 30s 3s/step - loss: 6.2318
Epoch 5/100
11/11 [==============================] - 31s 3s/step - loss: 5.5878
Epoch 6/100
11/11 [==============================] - 31s 3s/step - loss: 5.1504
Epoch 7/100
11/11 [==============================] - 31s 3s/step - loss: 4.8413
Epoch 8/100
11/11 [==============================] - 31s 3s/step - loss: 4.6171
Epoch 9/100
11/11 [==============================] - 31s 3s/step - loss: 4.4382
Epoch 10/100
11/11 [==============================] - 31s 3s/step - loss: 4.2898
Epoch 11/100
11/11 [==============================] - 31s 3s/step - loss: 4.1615
Epoch 12/100
11/11 [==============================] - 32s 3s/step - loss: 4.0487
Epoch 13/100
11/11 [================

## Over-sampling

In [22]:
sampler = RandomOverSampler(random_state=42)

X_train_os, y_train_os = sampler.fit_resample(X_train, y_train) 
print('Resampled dataset shape %s' % Counter(y_train_os)) 

Resampled dataset shape Counter({0: 1057, 1: 1057})


### Model 1

In [23]:
model = Sequential()
model.add(LSTM(
    50, activation='sigmoid', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(50, activation='sigmoid', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [25]:
model.fit(X_train_os, y_train_os, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
17/17 [==============================] - 6s 329ms/step - loss: 3.1067
Epoch 2/100
17/17 [==============================] - 6s 337ms/step - loss: 2.6443
Epoch 3/100
17/17 [==============================] - 6s 328ms/step - loss: 2.3104
Epoch 4/100
17/17 [==============================] - 5s 323ms/step - loss: 2.0626
Epoch 5/100
17/17 [==============================] - 6s 338ms/step - loss: 1.8793
Epoch 6/100
17/17 [==============================] - 6s 329ms/step - loss: 1.7428
Epoch 7/100
17/17 [==============================] - 6s 325ms/step - loss: 1.6398
Epoch 8/100
17/17 [==============================] - 6s 326ms/step - loss: 1.5603
Epoch 9/100
17/17 [==============================] - 6s 347ms/step - loss: 1.4975
Epoch 10/100
17/17 [==============================] - 6s 327ms/step - loss: 1.4471
Epoch 11/100
17/17 [==============================] - 6s 327ms/step - loss: 1.4052
Epoch 12/100
17/17 [==============================] - 6s 365ms/step - loss: 1.3683
Epoch 13/100


In [26]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 1s 52ms/step - loss: 0.7067


array([[0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.50901216],
       [0.509

### Model 4

In [30]:
model = Sequential()
model.add(LSTM(200, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(Dropout(0.1))  
model.add(LSTM(200, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [31]:
model.fit(X_train_os, y_train_os, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
17/17 [==============================] - 134s 3s/step - loss: 16.8643
Epoch 2/100
17/17 [==============================] - 49s 3s/step - loss: 13.8616
Epoch 3/100
17/17 [==============================] - 49s 3s/step - loss: 12.3750
Epoch 4/100
17/17 [==============================] - 49s 3s/step - loss: 11.6591
Epoch 5/100
17/17 [==============================] - 49s 3s/step - loss: 11.2934
Epoch 6/100
17/17 [==============================] - 49s 3s/step - loss: 11.0774
Epoch 7/100
17/17 [==============================] - 49s 3s/step - loss: 10.9232
Epoch 8/100
17/17 [==============================] - 49s 3s/step - loss: 10.7946
Epoch 9/100
17/17 [==============================] - 49s 3s/step - loss: 10.6777
Epoch 10/100
17/17 [==============================] - 49s 3s/step - loss: 10.5672
Epoch 11/100
17/17 [==============================] - 49s 3s/step - loss: 10.4611
Epoch 12/100
17/17 [==============================] - 49s 3s/step - loss: 10.3585
Epoch 13/100
17/17 [====

KeyboardInterrupt: 

In [32]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 3s 346ms/step - loss: 4.6476


array([[-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0.00703877],
       [-0

So it looks like over-sampling didn't help :(

In [ ]:
!nvidia-smi

### Model 5

In [35]:
model = Sequential()
model.add(LSTM(128, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(LSTM(128, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [36]:
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
11/11 [==============================] - 17s 1s/step - loss: 7.1326
Epoch 2/100
11/11 [==============================] - 15s 1s/step - loss: 5.6832
Epoch 3/100
11/11 [==============================] - 15s 1s/step - loss: 4.7034
Epoch 4/100
11/11 [==============================] - 15s 1s/step - loss: 4.0404
Epoch 5/100
11/11 [==============================] - 15s 1s/step - loss: 3.5787
Epoch 6/100
11/11 [==============================] - 15s 1s/step - loss: 3.2596
Epoch 7/100
11/11 [==============================] - 15s 1s/step - loss: 3.0345
Epoch 8/100
11/11 [==============================] - 16s 1s/step - loss: 2.8718
Epoch 9/100
11/11 [==============================] - 15s 1s/step - loss: 2.7510
Epoch 10/100
11/11 [==============================] - 15s 1s/step - loss: 2.6568
Epoch 11/100
11/11 [==============================] - 15s 1s/step - loss: 2.5803
Epoch 12/100
11/11 [==============================] - 15s 1s/step - loss: 2.5151
Epoch 13/100
11/11 [=================

In [37]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 2s 207ms/step - loss: 0.6243


array([[0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.21855006],
       [0.218

### Model 5 w/os

In [40]:
model = Sequential()
model.add(LSTM(128, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(LSTM(128, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [41]:
model.fit(X_train_os, y_train_os, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
17/17 [==============================] - 25s 1s/step - loss: 6.7897
Epoch 2/100
17/17 [==============================] - 23s 1s/step - loss: 5.0410
Epoch 3/100
17/17 [==============================] - 23s 1s/step - loss: 4.0339
Epoch 4/100
17/17 [==============================] - 23s 1s/step - loss: 3.4642
Epoch 5/100
17/17 [==============================] - 23s 1s/step - loss: 3.1340
Epoch 6/100
17/17 [==============================] - 24s 1s/step - loss: 2.9322
Epoch 7/100
17/17 [==============================] - 24s 1s/step - loss: 2.7932
Epoch 8/100
17/17 [==============================] - 24s 1s/step - loss: 2.6893
Epoch 9/100
17/17 [==============================] - 23s 1s/step - loss: 2.6033
Epoch 10/100
17/17 [==============================] - 23s 1s/step - loss: 2.5267
Epoch 11/100
17/17 [==============================] - 23s 1s/step - loss: 2.4558
Epoch 12/100
17/17 [==============================] - 23s 1s/step - loss: 2.3885
Epoch 13/100
17/17 [=================

In [42]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 2s 182ms/step - loss: 0.6943


array([[0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],
       [0.4949502],


### Model 1.2

In [43]:
model = Sequential()
model.add(LSTM(
    50, activation='sigmoid', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(50, activation='sigmoid', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [44]:
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
11/11 [==============================] - 6s 313ms/step - loss: 3.1964
Epoch 2/100
11/11 [==============================] - 3s 308ms/step - loss: 2.8004
Epoch 3/100
11/11 [==============================] - 3s 318ms/step - loss: 2.4956
Epoch 4/100
11/11 [==============================] - 4s 341ms/step - loss: 2.2466
Epoch 5/100
11/11 [==============================] - 4s 327ms/step - loss: 2.0482
Epoch 6/100
11/11 [==============================] - 3s 307ms/step - loss: 1.8891
Epoch 7/100
11/11 [==============================] - 4s 316ms/step - loss: 1.7646
Epoch 8/100
11/11 [==============================] - 3s 311ms/step - loss: 1.6602
Epoch 9/100
11/11 [==============================] - 3s 316ms/step - loss: 1.5692
Epoch 10/100
11/11 [==============================] - 4s 321ms/step - loss: 1.4965
Epoch 11/100
11/11 [==============================] - 4s 324ms/step - loss: 1.4369
Epoch 12/100
11/11 [==============================] - 4s 324ms/step - loss: 1.3875
Epoch 13/100


In [45]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 1s 54ms/step - loss: 0.5532


array([[0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.20958823],
       [0.209

In [58]:
model = Sequential()
model.add(LSTM(
    50, activation='sigmoid', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(50, activation='sigmoid', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [59]:
model.fit(X_train_os, y_train_os, epochs=300, verbose=1, batch_size=128)

Epoch 1/300
17/17 [==============================] - 9s 369ms/step - loss: 3.2422
Epoch 2/300
17/17 [==============================] - 6s 370ms/step - loss: 2.7200
Epoch 3/300
17/17 [==============================] - 6s 370ms/step - loss: 2.3442
Epoch 4/300
17/17 [==============================] - 6s 370ms/step - loss: 2.0761
Epoch 5/300
17/17 [==============================] - 6s 376ms/step - loss: 1.8831
Epoch 6/300
17/17 [==============================] - 6s 366ms/step - loss: 1.7435
Epoch 7/300
17/17 [==============================] - 6s 340ms/step - loss: 1.6408
Epoch 8/300
17/17 [==============================] - 6s 325ms/step - loss: 1.5637
Epoch 9/300
17/17 [==============================] - 6s 323ms/step - loss: 1.5006
Epoch 10/300
17/17 [==============================] - 6s 348ms/step - loss: 1.4506
Epoch 11/300
17/17 [==============================] - 6s 355ms/step - loss: 1.4140
Epoch 12/300
17/17 [==============================] - 6s 323ms/step - loss: 1.3769
Epoch 13/300


In [60]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 1s 54ms/step - loss: 0.6979


array([[0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],
       [0.5039608],


### Model 4.2

In [48]:
model = Sequential()
model.add(LSTM(200, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(Dropout(0.1))  
model.add(LSTM(200, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [49]:
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
11/11 [==============================] - 33s 3s/step - loss: 10.5423
Epoch 2/100
11/11 [==============================] - 30s 3s/step - loss: 7.9961
Epoch 3/100
11/11 [==============================] - 31s 3s/step - loss: 6.4224
Epoch 4/100
11/11 [==============================] - 31s 3s/step - loss: 5.4357
Epoch 5/100
11/11 [==============================] - 30s 3s/step - loss: 4.8218
Epoch 6/100
11/11 [==============================] - 30s 3s/step - loss: 4.4360
Epoch 7/100
11/11 [==============================] - 31s 3s/step - loss: 4.1834
Epoch 8/100
11/11 [==============================] - 31s 3s/step - loss: 4.0097
Epoch 9/100
11/11 [==============================] - 31s 3s/step - loss: 3.8825
Epoch 10/100
11/11 [==============================] - 31s 3s/step - loss: 3.7773
Epoch 11/100
11/11 [==============================] - 31s 3s/step - loss: 3.6868
Epoch 12/100
11/11 [==============================] - 31s 3s/step - loss: 3.6036
Epoch 13/100
11/11 [================

In [50]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 3s 323ms/step - loss: 0.6891


array([[0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.20466083],
       [0.204

In [61]:
model = Sequential()
model.add(LSTM(200, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(Dropout(0.1))  
model.add(LSTM(200, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [62]:
model.fit(X_train_os, y_train_os, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
17/17 [==============================] - 50s 3s/step - loss: 9.8821
Epoch 2/100
17/17 [==============================] - 47s 3s/step - loss: 6.8623
Epoch 3/100
17/17 [==============================] - 48s 3s/step - loss: 5.3686
Epoch 4/100
17/17 [==============================] - 47s 3s/step - loss: 4.6465
Epoch 5/100
17/17 [==============================] - 48s 3s/step - loss: 4.2778
Epoch 6/100
17/17 [==============================] - 48s 3s/step - loss: 4.0596
Epoch 7/100
17/17 [==============================] - 48s 3s/step - loss: 3.9050
Epoch 8/100
17/17 [==============================] - 49s 3s/step - loss: 3.7761
Epoch 9/100
17/17 [==============================] - 48s 3s/step - loss: 3.6594
Epoch 10/100
17/17 [==============================] - 48s 3s/step - loss: 3.5483
Epoch 11/100
17/17 [==============================] - 48s 3s/step - loss: 3.4419
Epoch 12/100
17/17 [==============================] - 48s 3s/step - loss: 3.3395
Epoch 13/100
17/17 [=================

In [63]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 3s 305ms/step - loss: 0.7039


array([[0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.49974126],
       [0.499

## Early Stopping

### Model 5

In [69]:
model = Sequential()
model.add(LSTM(128, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(LSTM(128, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [70]:
es = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128, validation_split=0.3, callbacks=[es])


Epoch 1/100
8/8 [==============================] - 14s 2s/step - loss: 7.3776 - val_loss: 6.6472
Epoch 2/100
8/8 [==============================] - 12s 1s/step - loss: 6.2244 - val_loss: 5.6264
Epoch 3/100
8/8 [==============================] - 12s 2s/step - loss: 5.3372 - val_loss: 4.9049
Epoch 4/100
8/8 [==============================] - 12s 2s/step - loss: 4.6828 - val_loss: 4.3465
Epoch 5/100
8/8 [==============================] - 12s 1s/step - loss: 4.1796 - val_loss: 3.9246
Epoch 6/100
8/8 [==============================] - 14s 2s/step - loss: 3.7991 - val_loss: 3.5998
Epoch 7/100
8/8 [==============================] - 12s 2s/step - loss: 3.5061 - val_loss: 3.3495
Epoch 8/100
8/8 [==============================] - 13s 2s/step - loss: 3.2825 - val_loss: 3.1576
Epoch 9/100
8/8 [==============================] - 14s 2s/step - loss: 3.1080 - val_loss: 3.0124
Epoch 10/100
8/8 [==============================] - 15s 2s/step - loss: 2.9726 - val_loss: 2.8928
Epoch 11/100
8/8 [===========

In [71]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 2s 190ms/step - loss: 1.6144


array([[0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.21184444],
       [0.21184433],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.21184525],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.2118442 ],
       [0.21184427],
       [0.211

In [72]:
model = Sequential()
model.add(LSTM(128, activation="relu", input_shape=(max_length, 1), return_sequences=True,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))  
model.add(LSTM(128, activation="relu", return_sequences=False,
              kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [73]:
es = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
model.fit(X_train_os, y_train_os, epochs=100, verbose=1, batch_size=128, validation_split=0.3, callbacks=[es])

Epoch 1/100
12/12 [==============================] - 23s 2s/step - loss: 7.0680 - val_loss: 6.5489
Epoch 2/100
12/12 [==============================] - 19s 2s/step - loss: 5.5926 - val_loss: 5.7582
Epoch 3/100
12/12 [==============================] - 19s 2s/step - loss: 4.6087 - val_loss: 4.7859
Epoch 4/100
12/12 [==============================] - 19s 2s/step - loss: 3.9531 - val_loss: 4.3299
Epoch 5/100
12/12 [==============================] - 18s 2s/step - loss: 3.5155 - val_loss: 3.9886
Epoch 6/100
12/12 [==============================] - 18s 2s/step - loss: 3.2196 - val_loss: 3.7645
Epoch 7/100
12/12 [==============================] - 19s 2s/step - loss: 3.0155 - val_loss: 3.5850
Epoch 8/100
12/12 [==============================] - 19s 2s/step - loss: 2.8688 - val_loss: 3.4322
Epoch 9/100
12/12 [==============================] - 19s 2s/step - loss: 2.7595 - val_loss: 3.3866
Epoch 10/100
12/12 [==============================] - 19s 2s/step - loss: 2.6731 - val_loss: 3.2940
Epoch 11/

In [74]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 2s 196ms/step - loss: 0.6244


array([[0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.2980025 ],
       [0.298

In [15]:
model = Sequential()
model.add(LSTM(
    30, activation='sigmoid', return_sequences=True, input_shape=(max_length, 1), kernel_regularizer="l2",
    recurrent_regularizer="l2", bias_regularizer="l2"
))
model.add(LSTM(40, activation='sigmoid', kernel_regularizer="l2", recurrent_regularizer="l2", bias_regularizer="l2"))
model.add(Dense(40))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss="binary_crossentropy")

In [13]:
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128)

Epoch 1/100
11/11 [==============================] - 5s 259ms/step - loss: 2.3678
Epoch 2/100
11/11 [==============================] - 3s 249ms/step - loss: 2.1332
Epoch 3/100
11/11 [==============================] - 3s 238ms/step - loss: 1.9393
Epoch 4/100
11/11 [==============================] - 3s 243ms/step - loss: 1.7842
Epoch 5/100
11/11 [==============================] - 3s 245ms/step - loss: 1.6516
Epoch 6/100
11/11 [==============================] - 3s 248ms/step - loss: 1.5428
Epoch 7/100
11/11 [==============================] - 3s 248ms/step - loss: 1.4497
Epoch 8/100
11/11 [==============================] - 3s 243ms/step - loss: 1.3732
Epoch 9/100
11/11 [==============================] - 3s 247ms/step - loss: 1.3081
Epoch 10/100
11/11 [==============================] - 3s 235ms/step - loss: 1.2535
Epoch 11/100
11/11 [==============================] - 3s 242ms/step - loss: 1.2062
Epoch 12/100
11/11 [==============================] - 3s 241ms/step - loss: 1.1669
Epoch 13/100


In [14]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 1s 41ms/step - loss: 0.5397


array([[0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.21605438],
       [0.216

In [16]:
es = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=128, validation_split=0.3, callbacks=[es])

Epoch 1/100
8/8 [==============================] - 5s 338ms/step - loss: 2.3777 - val_loss: 2.2537
Epoch 2/100
8/8 [==============================] - 2s 296ms/step - loss: 2.2067 - val_loss: 2.0994
Epoch 3/100
8/8 [==============================] - 2s 294ms/step - loss: 2.0587 - val_loss: 1.9595
Epoch 4/100
8/8 [==============================] - 2s 288ms/step - loss: 1.9269 - val_loss: 1.8378
Epoch 5/100
8/8 [==============================] - 2s 284ms/step - loss: 1.8105 - val_loss: 1.7333
Epoch 6/100
8/8 [==============================] - 2s 273ms/step - loss: 1.7119 - val_loss: 1.6402
Epoch 7/100
8/8 [==============================] - 2s 282ms/step - loss: 1.6256 - val_loss: 1.5570
Epoch 8/100
8/8 [==============================] - 2s 287ms/step - loss: 1.5443 - val_loss: 1.4893
Epoch 9/100
8/8 [==============================] - 2s 264ms/step - loss: 1.4791 - val_loss: 1.4233
Epoch 10/100
8/8 [==============================] - 2s 288ms/step - loss: 1.4181 - val_loss: 1.3677
Epoch 11/

In [17]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred

8/8 [==============================] - 0s 38ms/step - loss: 0.5995


array([[0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619311],
       [0.21619311],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.21619317],
       [0.216